## <div style="color: #E98E29">**1. Ler PDF completo** </div>

 - <s style="color: #92BB2D">CPFL</s>
 - <s style="color: #92BB2D">COPEL</s>
 - <s style="color: #92BB2D">CEMIG</s>
 - <s style="color: #92BB2D">EDP</s>
 - ELEKTRO
 - ENEL

---

## <div style="color: #E98E29">**2. Criar a base**</div>
- <s style="color: #92BB2D">Coletar Meses e ano(s)</s>
- <s style="color: #92BB2D">Coletar Consumos</s>
-  <s style="color: #92BB2D">Coletar Demandas</s>
- <s style="color: #92BB2D">Criar DataFrame</s>
- Adicionar a base de dados
<br>
<br>
---

## <s style="color: #92BB2D">**3. Não adicionar datas repetidas**</s>

<br>

## LER OS PDFS

In [1]:
import sys

classes = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\coleta\scripts\classes"
funcs = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\coleta\scripts\funcoes"
sys.path.append(classes)
sys.path.append(funcs)

In [57]:
import os
import contextlib
import regex as re
import pandas as pd
from f_database import *
from typing import Any, List, Tuple
from Fatura import Fatura

join_path = lambda y, x: os.path.join(y, x)
filename = lambda path, filetype: path.split("\\")[-1].split(filetype)[0]

PATH = r"Q:\APOIO ATENDIMENTO\Faturas EcoExp\Outubro"
FOLDERS = os.listdir(PATH)
PATHS = [{filename(x, ".pdf"): join_path(PATH, x)} for x in FOLDERS]

# SCRIPTS

## ADICIONAR IDS

In [ ]:
# ADICIONA NOVOS CLIENTES
df = []
ucs_df = df[["nome"]].drop_duplicates().reset_index(drop=True)
db_path = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\banco_de_dados\distribuidoras.db"
ucs_db = get_clientes_existentes(db_path)
ucs_nova = adiciona_novo_id_clientes(mantem_novos_clientes(ucs_df, ucs_db))
if not (ucs_nova.empty):
    print(f"Clientes a adicionar: {', '.join(ucs_nova.nome.values.tolist())}")
    if salva_novos_clientes(ucs_nova, db_path):
        print("Insercao de Novos Clientes ok")
        ucs_db = get_clientes_existentes(db_path)
    else:
        print("Erro ao salvar valores no banco")
else:
    print("Nenhuma UC nova")

consumos_pdf = criar_id_consumos(df_id=ucs_db, df_consumo=df.drop_duplicates(subset=["nome", "datas"]))
consumos_db = get_consumos_existentes(db_path)
novos_consumos = mantem_novos_consumos(
    novos_consumos=consumos_pdf, consumos_existentes=consumos_db
)
if not(novos_consumos.empty):
    status = salva_novos_consumos(novos_consumos=novos_consumos, db_path=db_path)

## DATAFRAME FINAL

In [61]:
from PyPDF2.errors import PdfReadError
from Identificador import Identificador


DISTRIBUIDORAS = {}

for info in PATHS:
    with contextlib.suppress(IndexError,TypeError, PdfReadError):
        path = info.get(*info.keys())
        DISTRIBUIDORA = Identificador(infos=info).main()(path)
        name = DISTRIBUIDORA.distribuidora
        print(name)
        if name not in DISTRIBUIDORAS.keys():
            DISTRIBUIDORAS[name] = []
        if name != None:
            if name in ["CPFL", "EDP", "COPEL", "CEMIG", "ENEL"]:
                DISTRIBUIDORA.main()
            DISTRIBUIDORAS[name].append(DISTRIBUIDORA)


# GRAVAR DADOS NO BANCO - DESENVOLVIMENTO

In [ ]:
CONSUMOS = []
DEMANDAS = []
for key in DISTRIBUIDORAS:
    if key in ["CPFL", "EDP", "COPEL", "CEMIG", "ENEL"]:
        for dis in DISTRIBUIDORAS[key]:
            consumo = dis.consumo.copy()
            consumo['nome'] = dis.nome.replace(" ", "_")
            consumo['uc'] = dis._caminho.split("\\")[-1][:-4]
            consumo['distribuidora'] = dis.distribuidora
            consumo['medida_consumo'] = dis.medida_consumo
            consumo['medida_demanda'] = dis.medida_demanda
            consumo['ths'] = dis.ths
            DEMANDAS.append(dis.demanda.copy())
            CONSUMOS.append(consumo)
consumos = pd.concat(CONSUMOS, ignore_index=True).set_index('datas')
demandas = pd.concat(DEMANDAS, ignore_index=True).set_index('datas')

In [16]:
infos = pd.concat([demandas, consumos], axis=1).drop_duplicates().fillna(-1).reset_index()
infos.consumo_ponta = infos.consumo_ponta.astype(float)
infos.demanda_ponta = infos.demanda_ponta.astype(float)
infos.demanda_fora_de_ponta = infos.demanda_fora_de_ponta.astype(float)
infos = remove_outliers(infos, "consumo_ponta", 0.9)

In [27]:
distribuidoras = infos.distribuidora.unique()
df = DataFrame(list(set(infos.groupby(["distribuidora", "nome", "uc"]).all().index)), columns=["distribuidora", "nome", "uc"])

In [28]:
db_path = r'C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\banco_de_dados\distribuidoras.db'
clientes = adiciona_novo_id_clientes(df.copy())
ucs_db = get_clientes_existentes(db_path)
novos_clientes = mantem_novos_clientes(df, ucs_db, col="nome")
# salva_novos_clientes(ucs_nova=novos_clientes, db_path=db_path)